In [5]:
import pandas as pd # Importing pandas library
import re # It is the "Regular Exprassion" from "Core Python"

# Loading dataset
df = pd.read_csv("Fault code dataset.csv", encoding="ISO-8859-1") # Due to the error like this ('utf-8' codec can't decode byte...),"encoding="ISO-8859-1" is used

# Function to extract fault codes
def extract_fault_codes(comment):
    if pd.isna(comment):  # Handle NaN values, "isna" is the function to handle NaN values
        return None
 # Regex to extract 2-4 digit numbers that are NOT preceded by "FC", "Fault Code", or "Code"
    matches = re.findall(r'(?:(?<!FC\s)(?<!Fault\sCode\s)(?<!Code\s))\b\d{2,5}\b', comment, re.IGNORECASE) # "Ignorecase for ignore the "upper and lowercase"
    return ', '.join(matches) if matches else None  # Return as comma-separated values
# Apply function to the "comment" column
df['fault_codes'] = df['Comments'].apply(extract_fault_codes)

# Function to extract fault codes (case insensitive)
def extract_fault_codes(text):
    if pd.isna(text):  # Handle NaN values
        return None
    codes = re.findall(r'\bFC\d+\b', text, re.IGNORECASE)  # Case-insensitive search
    return ', '.join(map(str.upper, codes)) if codes else None  # Convert to uppercase

column_name = "Comments"
df["Extracted Fault Codes Alphanumaric"] = df[column_name].apply(extract_fault_codes)

# Function to extract fault codes from text
def extract_fault_codes(text):
    if pd.isna(text):  # Handle NaN values
        return None
    # Regular expression to match different variations of fault codes
    pattern = r'\b(?:FC|F/C|FAULT CODE)\s*(\d+)\b'  # Matches FC, F/C, or Fault Code followed by numbers
    matches = re.findall(pattern, text, re.IGNORECASE)  # Case-insensitive search 
    return ', '.join(f'FC{code}' for code in matches) if matches else None  # Format as 

# Adjust column name to match the actual dataset
column_name = "Comments"  # Change to the actual column name
df["Extracted Fault Codes"] = df[column_name].apply(extract_fault_codes)

# Function to split multiple alphanumeric fault codes in a row
def split_multiple_fault_codes(text):
    if pd.isna(text):  # Handle NaN values
        return "", ""  # Return empty strings for missing data
    
    pattern = r'([A-Za-z]+)(\d+)'  # Match alphabets followed by numbers
    matches = re.findall(pattern, text)  # Find all occurrences
    
    if matches:
        alphabets_list, numbers_list = zip(*matches)  # Separate alphabets and numbers
        return ", ".join(alphabets_list), ", ".join(numbers_list)  # Convert to comma-separated strings
    
    return "", ""  # Return empty if no matches

# Apply function to the 'Extracted Fault Codes' column
df[['Alphabets', 'Numbers']] = df["Extracted Fault Codes"].apply(
    lambda x: pd.Series(split_multiple_fault_codes(str(x)))  # Ensure it's a string before processing
)

df.drop(columns=["Extracted Fault Codes"], inplace=True) # To extract the only numaric fault code 
df.drop(columns=["Extracted Fault Codes Alphanumaric"], inplace=True) # To extract only alphanumaric falut code
df["Merged_Fault_Codes"] = df["fault_codes"].astype(str) + ", " + df["Numbers"].astype(str) # merging these to falut code columns
df.drop(columns=["Numbers", "fault_codes", "Fault Codes"], inplace=True) # droping the unnecessary columns
df.drop(columns=["Alphabets"], inplace=True) # Droping this "Alphabets" column as well

# These block of codes are optional, can run and check the length of numbers of all the value from each row, for better identification of the fault code

# Extract all numbers, remove spaces, and find digit lengths
#all_numbers = df["Merged_Fault_Codes"].str.replace(" ", "").str.split(",")

# Flatten the list and get lengths of all numbers
#all_digit_lengths = [len(num) for sublist in all_numbers for num in sublist]

# Get min and max digit lengths
#min_digits = min(all_digit_lengths)
#max_digits = max(all_digit_lengths)

#print(f"Minimum digit length: {min_digits}")
#print(f"Maximum digit length: {max_digits}")

# Function to filter numbers with 3 to 5 digits

def filter_numbers(text):
    numbers = text.replace(" ", "").split(",")  # Split numbers and remove spaces
    filtered_numbers = [num for num in numbers if 3 <= len(num) <= 5]  # Keep only 3-5 digit numbers
    return ", ".join(filtered_numbers)  # Join back into a string

# Apply function to the column

df["Merged_Fault_Codes"] = df["Merged_Fault_Codes"].apply(filter_numbers)

# Function to keep only unique fault codes

def get_unique_fault_codes(text):
    numbers = text.replace(" ", "").split(",")  # Remove spaces and split numbers
    unique_numbers = sorted(set(numbers), key=numbers.index)  # Keep unique numbers in original order
    return ", ".join(unique_numbers)  # Join back into a string

# Apply function to create a new column with unique values

df["Unique_Fault_Codes"] = df["Merged_Fault_Codes"].apply(get_unique_fault_codes)

df.head(30)

,Comments,Merged_Fault_Codes,Unique_Fault_Codes
0,Stephen called for T/S assistance. Unit came i...,"120, 2023, 1682, 16825","120, 2023, 1682, 16825"
1,tech is troubleshooting fc2347 logged in the e...,"2023, 2347","2023, 2347"
2,Unit in with FC 1922 and complaint of coolant ...,"2023, 1922","2023, 1922"
3,***OOW***\n\nMatt calling has unit in T/S fc35...,"3545, 400, 2023, 2023, 3545, 3545, 3545","3545, 400, 2023"
4,Fault Code: 1921\nAftertreatment 1 Diesel Part...,"1921, 5911, 2023, 1921, 1921, 5911, 5911","1921, 5911, 2023"
5,"Customer complaint of CEL, tech verified compl...","2023, 2023, 3375, 1921, 1922, 2639, 3375, 3375...","2023, 3375, 1921, 1922, 2639"
6,EDS exhausted with no problem found. ECM image...,"2023, 2023, 3749, 3749","2023, 3749"
7,"Unit in with FC 3649, 3446. ECM image attached...","3446, 2554, 2023, 2554, 3649, 3135, 3135","3446, 2554, 2023, 3649, 3135"
8,"INITIAL COMPLAINT: CELF/C: 5655 , 3488 , 285...","5655, 3488, 285, 6339, 5655, 2023, 5655, 5655","5655, 3488, 285, 6339, 2023"
9,Customer complaint of CEl\ntech verified compl...,"2023, 555, 556, 556","2023, 555, 556"
